In [2]:
import pandas as pd
import nltk
import pymorphy2
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim import corpora, models
import re
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import time
import time
from tqdm import tqdm

In [3]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    PER,
    NamesExtractor,
    Doc
)

segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)
names_extractor = NamesExtractor(morph_vocab)

In [12]:
data = pd.read_json('data\\prof_rost.json')

In [13]:
start_time = time.time()
texts_df = data["text"].fillna("") \
    .astype("string").replace(to_replace='\n', value=' ',regex=True) \
    .replace(to_replace='\.', value=' ',regex=True).apply(lambda x: re.sub('[^# а-яА-Я]', '', x))
print("--- %s seconds ---" % (time.time() - start_time))

--- 37.98272395133972 seconds ---


In [14]:
def separate_hashtags(text):
    pure_text = []
    hashtags = []
    for word in text.split(' '):
        if len(word) > 0:
            #f word[0] != '#':
            if '#' not in word:
                pure_text.append(word)
           # else:
            #    hashtags.append(word)
    return pure_text #, hashtags


import pymorphy2
morph = pymorphy2.MorphAnalyzer()
def natasha_lem(text_list):
    
    lem_text = []
    for word in text_list:
        p = morph.parse(word)[0]
        lem_text.append(p.normal_form)
    with open(r'normalized_texts.txt', 'a') as fp:
        fp.write(' '.join(lem_text))
        fp.write('\n')
    return lem_text
        

In [16]:
start_time = time.time()
text_pure= texts_df.progress_apply(separate_hashtags)

print("--- %s seconds ---" % (time.time() - start_time))
text_pure

  0%|          | 0/474835 [00:00<?, ?it/s]

--- 71.18436193466187 seconds ---


0         [Главное, основные, качеств, предпринимателя, ...
1         [Сегодня, улыбнуло, когда, увидела, Текст, на,...
2         [Удачного, и, счастливого, нам, дня, Для, того...
3         [Не, прошло, и, пару, месяцев, с, тех, пор, ка...
4         [Тренинг, личностного, роста, Развитие, мелкой...
                                ...                        
474830    [ЭТО, ТО, ЧТО, НУЖНО, КАЖДОМУ, Подарок, которы...
474831    [ВНИМАНИЕ, МОДЕЛЬНОЕ, АГЕНТСТВО, ТАТЬЯНЫ, ЖИЛИ...
474832    [Анна, как, выбрать, чем, заниматься, Вопрос, ...
474833    [Зачастую, мы, идем, к, холодильнику, от, нече...
474834    [пройдет, курс, педикюр, идеальное, покрытие, ...
Name: text, Length: 474835, dtype: object

In [18]:
start_time = time.time()
lem_texts= text_pure.progress_apply(natasha_lem)
print("--- %s seconds ---" % (time.time() - start_time))

  0%|          | 0/474835 [00:00<?, ?it/s]

--- 50005.58027887344 seconds ---


In [2]:
with open('normalized_texts.txt') as f:
    lines = f.read().splitlines()
len(lines)

474835

In [3]:
lines[0]

'главное основной качество предприниматель это скорость принятие решение эксперт ли я очень любить учиться однако при выбор обучение я ни раз сталкиваться с тем что больший часть курс вода или то что я уже знать эксперт действительно иметь большой опыт в тот что преподата но являться плохой оратор и поэтому он речь путанна как и подача материал в общий если ты много знать это ещ не значит что ты хороший учитель некоторый эксперт восхищаться сотня человек но они банально я не заходить например рыбак не смочь заинтересовать я ни на один из офлайн встреча хотя он копирайтер писать просто божественный продавать текст на он рекламный пост и самый обидный когда эксперт не иметь практически опыт и образование но прошлый курс как стать эксперт и стать они не быть озвучивать имя но есть у я один знакомый у который разориться собственный бизнес но он прошлый обучение как разбирать чужой бизнес и теперь делать деньга на это ну или пытаться поэтому я перед покупка любой курс стараться как можно бо

In [4]:
rep = {"личный рост": "личный_рост",
       "профессиональный рост": "профессиональный_рост",
       "личностный рост": "личностный_рост",
       "развитие себя" : "развитие_себя",
       "улучшение себя" : "улучшение_себя",
       "профессиональное развитие" : "профессиональное_развитие",
       "личностное развитие" : "личностное_развитие",
       "карьерный рост" : "карьерный_рост",
       "карьерное развитие" : "карьерное_развитие",
       "развитие карьеры" : "развитие_карьеры",
       "работа над собой" : "работа_над_собой",
       "работа с собой": "работа_с_собой"} # cooloc to glue

rep = dict((re.escape(k), v) for k, v in rep.items()) 
pattern = re.compile("|".join(rep.keys()))
def substring_change(text):
    text = pattern.sub(lambda m: rep[re.escape(m.group(0))], text)
    with open(r'norm_text_colloc.txt', 'a') as fp:
        fp.write(text)
        fp.write('\n')
    return text 

In [5]:
for line in tqdm(lines):
    substring_change(line)

100%|██████████| 474835/474835 [02:20<00:00, 3386.99it/s]
